In [ ]:
'''
This Notebook will request the DailyMed Inital call dump to MongoDB with Next call Steps
1. Call base API for Product Collection
2. Save Base Response in a collection
3. Save Next Response in a collection
4. Iterate Response for Product in Base Response Collection
5. Validate Against DailyMed Schema.
6. Update and Log Schema Node in Question
7. Conver XML Response to JSON as per DailyMed Schema Specification
8. Save to DailyMed Product Response Collection.
'''

In [32]:
#-------- Import for Notebook ---------
import os
import requests
from pymongo import MongoClient
import pandas as pd
import random
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from IPython.display import display
#------- End Import -------------------

In [33]:
# ------- Global Var of Notebook ------

cde_pth =os.getcwd()

DAILYMED_URL = "https://dailymed.nlm.nih.gov/dailymed/services/v2"

DB_HOST = "localhost"

DB_PORT = 27017

DB_NAME ="dailymed"

NDC_COLXN ="spls"

NDC_COLXN_NEXT ="spls_next"

In [34]:

# Styling notebook
from IPython.core.display import HTML
# Styling notebook
def css_styling():
    styles = open( cde_pth + "/custom.css", "r").read()
    HTML(''.format(styles))
    #return HTML(styles)
css_styling()

HTML("""
<style>
.renderjson a              { text-decoration: none !important; }
.renderjson .disclosure    { color: crimson;
                             font-size: 100%; }
.renderjson .syntax        { color: grey; }
.renderjson .string        { color: red; }
.renderjson .number        { color: cyan; }
.renderjson .boolean       { color: plum; }
.renderjson .key           { color: lightblue; }
.renderjson .keyword       { color: lightgoldenrodyellow; }
.renderjson .object.syntax { color: lightseagreen; }
.renderjson .array.syntax  { color: lightsalmon; }
/* unvisited link */

.renderjson a:link {
  color: blue;
}

/* visited link */
.renderjson a:visited {
  color: green;
}

/* mouse over link */
.renderjson a:hover {
  color: hotpink;
}

/* selected link */
.renderjson a:active {
  color: blue;
}
</style>
""")
#----------------------------------------------------------    
# Class utilized for Display At Notebook
'''
    Class utilized for Display At Notebook
    https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook
    https://github.com/caldwell/renderjson
'''
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):        
        display_html('<div id="{}" style="height: 250px; width:100%;" class="renderjson"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_icons("+", "-");
          renderjson.set_show_to_level(1);
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True) 


import json
from datetime import datetime
from typing import Any

from bson import ObjectId


class MongoJSONEncoder(json.JSONEncoder):
    def default(self, o: Any) -> Any:
        if isinstance(o, ObjectId):
            return str(o)
        if isinstance(o, datetime):
            return str(o)
        return json.JSONEncoder.default(self, o)
    
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            

class MongoDBClient:
    def __init__(self, db_hostname=None,db_port=None, db_name=None):
        if db_hostname is None:
            self.hostname =DB_HOST
        else:
            self.hostname = db_hostname
        if db_port is None:
            self.port = DB_PORT
        else:
            self.port = db_port
        if db_name is None:
            self.dbName = DB_NAME
        else:
            self.dbName = db_name
    
    def create_client(self):
        client = MongoClient(self.hostname,self.port)        
        self.client = client
        return client
    
    def get_list_databases(self):
        if self.client is None:
            self.client = self.create_client()
            
        return self.client.list_databases()
        
    def get_dailymed_db(self,dbName=None):
        client = self.create_client()
        db = None
        if dbName is None:
            db = client[self.dbName]
        else:
            db = client[dbName]
        return db
    
    def get_list_collection(self, dbName=None):
        if self.client is None:
            self.client = self.create_client()
            
        db = self.get_dailymed_db(dbName)
        clxns = None
        if db is not None:
            clxns = db.list_collection_names()
        return clxns
            
class BaseJsonRequest:
    def __init__(self,api_url=None):
        self.api_url = api_url
    
    def get_json_request(self,api_url=None,path_suffix=None, params=None):
        if api_url is None:
            api_url = self.api_url
            
        if path_suffix is not None:
            api_url = api_url+"/" + path_suffix
        print("Fetching URL : "+api_url)
        r = requests.get(url = api_url, params = params)
        return r.json()

In [35]:
def testRequest():
    print("~~~~~~~~~~~ Test Run Classes START ~~~~~~~~~~~~~~~")
    br = BaseJsonRequest(api_url=DAILYMED_URL)
    #print(br.api_url)
    respo = br.get_json_request(None,"spls.json",{})
    return respo

In [36]:
respo = testRequest()
RenderJSON(respo)

~~~~~~~~~~~ Test Run Classes START ~~~~~~~~~~~~~~~
Fetching URL : https://dailymed.nlm.nih.gov/dailymed/services/v2/spls.json


In [37]:
for entry in respo["data"]:    
    DAILYMED_PRODUCT_URL = DAILYMED_URL+"/spls/"+entry["setid"]+".xml"
    print(DAILYMED_PRODUCT_URL)

https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/010290af-81f4-0037-e063-6394a90a4638.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/017bc6a3-5b3d-2381-e063-6394a90abece.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/0269762a-221b-5ea2-e063-6294a90ac0d4.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026998ed-ed38-48b9-e063-6394a90a20c7.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/0269c5b1-10cf-d52b-e063-6294a90af99c.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026a7219-78ba-48c7-e063-6394a90ad736.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026aefd6-bb80-1ffd-e063-6394a90a1262.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026b8919-31f8-e708-e063-6394a90ad401.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026b9b30-ef08-bf2e-e063-6394a90a54a6.xml
https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/026bb47b-5b4a-2324-e063-6394a90a5903.xml
https://dailymed.nlm.nih.gov/dailymed/se

In [38]:
r1 = requests.get(url = "https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/010290af-81f4-0037-e063-6394a90a4638.xml")

In [39]:
#print(r1.content)
bs_xml = BeautifulSoup(r1.content, 'xml')

In [40]:
print(bs_xml)

<?xml version="1.0" encoding="utf-8"?>
<?xml-stylesheet href="https://www.accessdata.fda.gov/spl/stylesheet/spl.xsl" type="text/xsl"?><document xmlns="urn:hl7-org:v3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="urn:hl7-org:v3 https://www.accessdata.fda.gov/spl/schema/spl.xsd">
<id root="01029181-9b54-99dd-e063-6294a90a58d5"/>
<code code="34391-3" codeSystem="2.16.840.1.113883.6.1" displayName="HUMAN PRESCRIPTION DRUG LABEL"/>
<title>These highlights do not include all the information needed to use CARVEDILOL TABLETS safely and effectively. See full prescribing information for CARVEDILOL TABLETS.
 <br/>
<br/>
CARVEDILOL tablets, for oral use.
 <br/>
Initial U.S. Approval: 1995
</title>
<effectiveTime value="20230801"/>
<setId root="010290af-81f4-0037-e063-6394a90a4638"/>
<versionNumber value="1"/>
<author>
<time/>
<assignedEntity>
<representedOrganization>
<id extension="030620888" root="1.3.6.1.4.1.519.1"/>
<name>QPharma Inc</name>
<assignedEntity>
<assign